In [8]:
import os
from enum import Enum
from typing import Any, Dict, List

from openai import OpenAI, AsyncOpenAI
import pandas as pd
import pprint
from pydantic import BaseModel, Extra, Field
import tiktoken

from config import settings

In [4]:
openai_client = OpenAI(api_key=settings.openai_api_key)

In [5]:
tokenizer = tiktoken.encoding_for_model("gpt-4o")

In [6]:
def calculate_tokens(text):
    return len(tokenizer.encode(text))
calculate_tokens("hello world")

2

# TVTropes Dataset
* https://github.com/dhruvilgala/tvtropes

In [2]:
# tropes table
tropes_fname = "story/TVTropesData/tropes.csv"
tropes_fpath = os.path.join(settings.data_dir, tropes_fname)

tropes_df = pd.read_csv(tropes_fpath)
print(tropes_df.shape, tropes_df.columns)

(30984, 4) Index(['Unnamed: 0', 'TropeID', 'Trope', 'Description'], dtype='object')


In [3]:
tropes_df.head()

,Unnamed: 0,TropeID,Trope,Description
0,0,t00001,AbandonedArea,\nAbandoned places make good settings for fict...
1,1,t00002,AbandonedCatchphrase,Catchphrases are a great and simple way to hel...
2,2,t00003,AbandonedHospital,The creepy abandoned hospital/mental instituti...
3,3,t00004,AbandonedHospitalAwakening,"An Abandoned Hospital Awakening is, as the nam..."
4,4,t00005,AbandonedInfoPage,When a work is getting more and more complicat...


In [9]:
idx = 1
print("TROPE:")
print(tropes_df.iloc[idx]["Trope"])
print("DESC:")
print(tropes_df.iloc[idx]["Description"].strip())

TROPE:
AbandonedCatchphrase
DESC:
Catchphrases are a great and simple way to help a character's image. But sometimes, catchphrases don't stick. Perhaps the writers were experimenting with one early on, but eventually decided against it. Or perhaps a catchphrase was done to death and is no longer funny. Or sometimes, a character simply evolves and the catchphrase no longer fits the character.
Whatever the reason, this trope is whenever an early catchphrase is dropped or becomes used seldom. This is a catchphrase specific sub-trope of Early Installment Weirdness, and frequently a result of Characterization Marches On and Character Development. The closest inverse to this would be Flanderization (wherein a character's catchphrase acts as a quirk that takes over their personality). Unrelated to Subverted Catch Phrase.


In [11]:
# film-tropes table (tropes in films)
film_tropes_fname = "story/TVTropesData/film_tropes.csv"
film_tropes_fpath = os.path.join(settings.data_dir, film_tropes_fname)

film_tropes_df = pd.read_csv(film_tropes_fpath)
print(film_tropes_df.shape, film_tropes_df.columns)

(751594, 6) Index(['Unnamed: 0', 'Title', 'Trope', 'Example', 'trope_id', 'title_id'], dtype='object')


In [15]:
film_title_ids = film_tropes_df.title_id.unique()
len(film_title_ids), film_title_ids[:5]

(17019, array(['f0', 'f1', 'f2', 'f3', 'f4'], dtype=object))

In [17]:
title_rows = film_tropes_df[film_tropes_df.title_id=='f0']
print(title_rows.shape)

(35, 6)


In [22]:
print("Title:",title_rows.iloc[0]["Title"])
# for i in range(title_rows.shape[0]):
for i in range(5):
    row = title_rows.iloc[i]
    print(row["Trope"])
    print(row["Example"].strip())
    print('-'*30)

Title: ABBATheMovie
MsFanservice
The concert segments make it clear that Agnetha and Frida's looks play a big part of the band's act; the "Get on the Carousel"/"I'm a Marionette" segment has them in form-fitting leotards that have them cut very shapely figures, and Frida at points also wears very short shorts (such as in one performance of "Why Did It Have to Be Me?"), leading to some sensationalist headlines hoping for a  Wardrobe Malfunction . And that's not even counting the repeated mentions about Agnetha's bottom.
------------------------------
InsistentTerminology
The radio station manager that tasks Ashley with delivering an interview with the band insists that it's not an interview, it's a  dialogue , and when Ashley asks if he wants him to do a documentary, he says that it's not a documentary, it's an  event .
------------------------------
TheIngenue
A rare male example; when Benny reads one newspaper reporting about the band asking for a "kinky bed", he ask what "kinky" mean